# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

# Линейная регрессия, регуляризация, гиперпараметр, полиномиальная регрессия

In [176]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

data = pd.read_csv('result.csv', sep = ',')
data = data.drop(columns = ['Unnamed: 0'])
data

,price,area,latitude,longitude,Bedrooms,Bathrooms,Balcony,parking,Lift,Ready to Move,Under Construction,New Property,Resale,Furnished,Semi-Furnished,Unfurnished,Flat,Individual House
0,22400000.0,629.0,19.032800,72.896357,2.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
1,35000000.0,974.0,19.032800,72.896357,3.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
2,31700000.0,968.0,19.085600,72.909277,3.0,3.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
3,18700000.0,629.0,19.155756,72.846862,2.0,2.0,2.0,2.0,2.0,1,0,1,0,0,0,1,1,0
4,13500000.0,1090.0,19.177555,72.849887,2.0,2.0,0.0,0.0,0.0,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,19500000.0,810.0,19.138320,72.810020,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,1,0,1,0
6251,22000000.0,1400.0,19.221920,72.854250,3.0,3.0,1.0,1.0,1.0,1,0,0,1,0,0,1,1,0
6252,20000000.0,750.0,19.144320,72.824111,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,0,1,1,0
6253,11000000.0,700.0,19.047201,72.872225,2.0,2.0,0.0,0.0,0.0,1,0,0,1,1,0,0,1,0


In [377]:
y = data['price']
X = data.drop(['price'], axis = 1)

In [213]:
y

0       22400000.0
1       35000000.0
2       31700000.0
3       18700000.0
4       13500000.0
           ...    
6250    19500000.0
6251    22000000.0
6252    20000000.0
6253    11000000.0
6254    15000000.0
Name: price, Length: 6255, dtype: float64

In [214]:
X

,area,latitude,longitude,Bedrooms,Bathrooms,Balcony,parking,Lift,Ready to Move,Under Construction,New Property,Resale,Furnished,Semi-Furnished,Unfurnished,Flat,Individual House
0,629.0,19.032800,72.896357,2.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
1,974.0,19.032800,72.896357,3.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
2,968.0,19.085600,72.909277,3.0,3.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
3,629.0,19.155756,72.846862,2.0,2.0,2.0,2.0,2.0,1,0,1,0,0,0,1,1,0
4,1090.0,19.177555,72.849887,2.0,2.0,0.0,0.0,0.0,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,810.0,19.138320,72.810020,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,1,0,1,0
6251,1400.0,19.221920,72.854250,3.0,3.0,1.0,1.0,1.0,1,0,0,1,0,0,1,1,0
6252,750.0,19.144320,72.824111,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,0,1,1,0
6253,700.0,19.047201,72.872225,2.0,2.0,0.0,0.0,0.0,1,0,0,1,1,0,0,1,0


In [215]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [142]:
lr = LinearRegression().fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred

array([27177670.20618057, 38845378.00874138, 58187894.17677498, ...,
       18345903.54318237, 23899884.27685452, 11439649.96230125])

In [143]:
y_pred.shape

(1251,)

In [144]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr.score(X_test, y_test)}')

MAE: 10231528.734740173
MSE: 326239809616389.3
RMSE: 18062109.777553376
MAPE: 0.7513574051424674
R^2: 0.640183993102154


In [145]:
len(lr.coef_)
lr.coef_

array([ 2.13771508e+04, -7.90092497e+07, -7.28394540e+07,  6.84026190e+05,
        7.30302657e+06, -1.33150346e+05, -1.33150346e+05, -1.33150346e+05,
       -2.09529614e+06,  2.09529614e+06,  7.06370378e+04, -7.06370378e+04,
        1.54892456e+06, -6.43694409e+05, -9.05230150e+05,  5.12404739e+03,
       -5.12404739e+03])

In [146]:
#L1
ridge = Ridge(alpha = 0.7).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 10206923.167395014
MSE: 326711873409971.7
RMSE: 18075172.845922433
MAPE: 0.7480307584217453
R^2: 0.6396633450874081


array([ 2.14085214e+04, -7.79368189e+07, -7.00100045e+07,  6.66352448e+05,
        7.34105952e+06, -1.38645817e+05, -1.38645817e+05, -1.38645817e+05,
       -1.88974525e+06,  1.88974525e+06,  7.43382611e+04, -7.43382611e+04,
        1.57313903e+06, -6.61587100e+05, -9.11551932e+05,  2.38109620e+04,
       -2.38109620e+04])

In [147]:
parameters = {'alpha' : np.arange(0, 1, 0.1)}
ridge_optimal = GridSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

{'alpha': 0.7000000000000001}

In [148]:
#L2
lasso = Lasso(alpha = 0.9).fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

MAE: 10231526.730257409
MSE: 326239851789442.1
RMSE: 18062110.944998708
MAPE: 0.7513571381971351
R^2: 0.6401839465887028


array([ 2.13771545e+04, -7.90091713e+07, -7.28391936e+07,  6.84024331e+05,
        7.30302762e+06, -3.99452325e+05, -2.86669284e-10, -8.74092218e-11,
       -4.18909164e+06,  2.18349000e-09,  1.41272826e+05, -6.55732489e-10,
        2.42368344e+06,  2.31061052e+05, -3.04680649e+04,  1.02361243e+04,
       -0.00000000e+00])

In [149]:
parameters = {'alpha' : np.arange(1, 50, 1)}
ridge_optimal = GridSearchCV(Lasso(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

{'alpha': 49}

# Полиномиальная регрессия

In [150]:
lr = make_pipeline(PolynomialFeatures(2), LinearRegression()).fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [151]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
lasso.coef_

MAE: 8828824.762505425
MSE: 234073072187048.38
RMSE: 15299446.79349709
MAPE: 0.6797632683587348


array([ 2.13771545e+04, -7.90091713e+07, -7.28391936e+07,  6.84024331e+05,
        7.30302762e+06, -3.99452325e+05, -2.86669284e-10, -8.74092218e-11,
       -4.18909164e+06,  2.18349000e-09,  1.41272826e+05, -6.55732489e-10,
        2.42368344e+06,  2.31061052e+05, -3.04680649e+04,  1.02361243e+04,
       -0.00000000e+00])

# Самостоятельное выполнение

# Метрики

In [373]:
def mae(y, y_pred):
    return abs(y-y_pred).mean()

def mse(y, y_pred):
    return ((y-y_pred)**2).mean()

def rmse(y, y_pred):
    return sqrt(((y-y_pred)**2).mean())

def mape(y, y_pred):
    return abs((y-y_pred)/y).mean()

# def R2(y, y_pred):
#     return 1 - ((y-y_pred)**2).mean() / ((y-y.mean())**2).mean()
def R2(y_r, y_p):
        return 1 - (np.sum((y_p-np.mean(y_r))**2))/(np.sum((y_r-np.mean(y_r))**2))

In [153]:
print(mae(y_test, y_pred))
print(mse(y_test, y_pred))
print(rmse(y_test, y_pred))
print(mape(y_test, y_pred))
print(R2(y_test, y_pred))

8828824.762505425
234073072187048.38
15299446.79349709
0.4620781010097493
0.7418364170341771


# Линейная регрессия с регуляризацией

In [367]:
class LinearRegression:
    
    def grad(self, w, n, alpha):
        buf = [0 for i in range(len(w))]
        for i in range(len(self.X)):
            buf += (self.X[i]*w - self.y[i])*self.X[i][n] + alpha*np.sign(w) #np.linalg.norm(w)
        return buf
    
    def fit(self, X, y, eta = 1e-8, alpha = 0.5):
        self.X = X.values
        self.y = y.values
        self.X = np.hstack((np.ones((self.X.shape[0], 1)), self.X))
        self.w = np.zeros(self.X.shape[1])
        for i in range(self.X.shape[1]):
            self.w = self.w - (2 * eta / self.X.shape[0]) * self.grad(self.w, i, alpha)
    def predict(self, X):
        X = X.values
        return np.sum(X * self.w[1:],axis = 1)

In [368]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
mae(y_test, y_pred)

27168730.547832623

In [374]:
print(mae(y_test, y_pred))
print(mse(y_test, y_pred))
print(rmse(y_test, y_pred))
print(mape(y_test, y_pred))
print(R2(y_test, y_pred))

27168730.547832623
1620513363899729.5
40255600.40416401
0.9303534006885364
0.18551592483966306


In [370]:
print(f'R^2: {ridge.score(X_test, y_test)}')

R^2: 0.6396633450874081


In [379]:
y.mean()

26841873.70103917